In [1]:
import pandas as pd
from keras.layers import LSTM, Input, Bidirectional, TimeDistributed, Dropout, Dense, Activation
from keras.models import Model
from keras.utils.np_utils import to_categorical
import tensorflow as tf
import numpy as np

Using TensorFlow backend.


## Model

In [2]:
def get_model(input_shape, output_shape, lr = 0.00001):
    inputs = Input(shape=input_shape)
    X = Dropout(0.5)(inputs)
    X = Dense(4096, activation='relu')(X)
    X = Dropout(0.5)(X)
    X = Dense(2048, activation='relu')(X)
    X = Dense(2048, activation='relu')(X)
    X = Bidirectional(LSTM(1024, return_sequences=True))(X)
    outputs = TimeDistributed(Dense(output_shape, activation='softmax'))(X)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=tf.train.AdamOptimizer(lr),
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy'],
    )
    
    return model

## Data

In [3]:
def get_featuers_by_name(net_name):
    features_train = np.load('./features/' + net_name + '/features_train.npy')
    features_dev = np.load('./features/'+ net_name +'/features_dev.npy')
    return features_train, features_dev

def get_features(names):
    features_train = []
    features_dev = []
    for name in names:
        train, dev = get_featuers_by_name(name)
        features_train.append(train)
        features_dev.append(dev)
    
    features_train = np.concatenate(features_train, axis=1)
    features_dev = np.concatenate(features_dev, axis=1)
    return np.stack([features_train, features_train], axis=1), np.stack([features_dev, features_dev], axis=1)

def get_classes_by_name(name):
    classes_train = np.load('./features/res_net/classes_train_' + name + '.npy')
    classes_train = to_categorical(classes_train)
    classes_dev = np.load('./features/res_net/classes_dev_' + name + '.npy')
    classes_dev = to_categorical(classes_dev)
    return classes_train, classes_dev

def get_classes(names):
    classes_train = []
    classes_dev = []
    num_classes = []
    for name in names:
        train, dev = get_classes_by_name(name)
        classes_train.append(train)
        classes_dev.append(dev)
        num_classes.append(train.shape[1])
        
    max_num_classes = np.max(num_classes)
    
    padded_classes_train = []
    padded_classes_dev = []
    for train, dev in zip(classes_train, classes_dev):
        padded_classes_train.append(
            np.pad(
                train, 
                pad_width=((0, 0), (0, max_num_classes - train.shape[1])),
                mode='constant',
                constant_values=((0,0), (0,0)),
            )
        )
        
        padded_classes_dev.append(
            np.pad(
                dev, 
                pad_width=((0, 0), (0, max_num_classes - dev.shape[1])),
                mode='constant',
                constant_values=((0,0), (0,0)),
            )
        )
        
    return np.stack(padded_classes_train, axis=1), np.stack(padded_classes_dev, axis=1)

In [4]:
features_train, features_dev = get_features([
    'res_net',
    'inception',
    'xception'
])

In [5]:
print('Train shape: ', features_train.shape, 'Dev shape:', features_dev.shape)

Train shape:  (12872, 2, 6144) Dev shape: (3313, 2, 6144)


In [6]:
classes_train, classes_dev = get_classes(['make', 'model'])

In [7]:
print('Train classes shape: ', classes_train.shape, 'Dev classes shape:', classes_dev.shape)

Train classes shape:  (12872, 2, 167) Dev classes shape: (3313, 2, 167)


## Training

In [8]:
model = get_model((features_train.shape[1:]), classes_train.shape[2])

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2, 6144)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 6144)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 2, 4096)           25169920  
_________________________________________________________________
dropout_2 (Dropout)          (None, 2, 4096)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 2, 2048)           8390656   
_________________________________________________________________
dense_3 (Dense)              (None, 2, 2048)           4196352   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 2, 2048)           25174016  
__________

In [ ]:
model.fit(
    features_train,
    classes_train,
    batch_size=8,
    epochs=50,
    validation_data=(features_dev, classes_dev)
)

Train on 12872 samples, validate on 3313 samples
Epoch 1/50
